In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    port=80,
    username='mail_id@rapido.bike'
)

In [2]:
start_date = '2022-10-01'
end_date = '2022-10-31'

In [3]:
auto = f"""

WITH base_month_customer AS 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d') AS month,
    customer_id,
    --MAX(auto_lifetime_last_ride_date) 
    auto_lifetime_last_ride_date,
    auto_lifetime_first_ride_date,
    --MAX(auto_lifetime_rides) 
    auto_lifetime_rides

FROM datasets.iallocator_customer_segments
WHERE 
    run_date BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date)) , '%Y-%m-%d') 
    AND run_date = DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d')
    AND (auto_lifetime_rides > 0 or auto_lifetime_rides is not null or auto_lifetime_last_ride_date is not null)
--GROUP BY 1,2
),

base_month_segment AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    customer_id,
    CASE
    WHEN auto_lifetime_rides IS NULL OR DATE(month) = DATE(auto_lifetime_first_ride_date) THEN 'LTR =0'
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 THEN 'LTR 1-4'
    WHEN (auto_lifetime_rides BETWEEN 5 AND 20) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 5-20'
    WHEN (auto_lifetime_rides BETWEEN 21 AND 50) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 21-50'
    WHEN (auto_lifetime_rides > 50) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 50+'
    WHEN auto_lifetime_rides >= 5 THEN 'LTR >=5'
    ELSE 'LTR =0'
    END ltr_auto,
    CASE 
    WHEN auto_lifetime_last_ride_date IS NULL OR DATE(month) = DATE(auto_lifetime_first_ride_date) THEN 'LTR=0'
    
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 and auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '7' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '0' DAY) THEN '1. 1_7_DAYS'
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 and auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '14' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '8' DAY) THEN '2. 8_14_DAYS'
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 and auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '21' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '15' DAY) THEN '3. 15_21_DAYS'
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 and auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '22' DAY) THEN '4. 22_30_DAYS'
    
    WHEN auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN '5. LAST_30_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '45' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '6. 31_45_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '60' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '46' DAY) THEN '7. 46_60_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '61' DAY) THEN '8. 61_90_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '9. 91_180_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '99. 181_365_DAYS'
    WHEN auto_lifetime_last_ride_date < (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) THEN '999. 365_ABOVE'
    ELSE 'LTR=0'
    END recency_auto
FROM base_month_customer
),

base_month_total AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    COUNT(DISTINCT customer_id) existing_custr_total 
FROM base_month_customer
GROUP BY 1
),

customer_rf_daily_kpi AS
(
SELECT 
    day,
    customerid,
    rr_sessions_unique_daily,
    net_rides_daily,
    ao_sessions_unique_daily,
    fe_sessions_unique_daily,
    discount_daily,
    subtotal_daily
        
FROM datasets.customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
    AND service_name = 'Auto'
    AND customerid IS NOT NULL
),

rr_net AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customerid as customer_id,
    SUM(rr_sessions_unique_daily) rr,
    SUM(net_rides_daily) net_rides,
    SUM(discount_daily) burn,
    SUM(subtotal_daily) subtotal
FROM customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
GROUP BY 1,2
),

max_daily_ao AS
(
SELECT 
        day,
        customer_id,
        SUM(ao) AS ao,
        SUM(fe) AS fe
FROM
    (
    SELECT 
        customerid as customer_id,
        day,
        max(ao_sessions_unique_daily) AS ao,
        max(fe_sessions_unique_daily) AS fe
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IS NOT NULL
    GROUP BY 1,2
    )
GROUP BY 1,2
),

monthly_ao AS
(
SELECT  
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customer_id,
    SUM(ao) AS ao,
    SUM(fe) AS fe
FROM max_daily_ao
GROUP BY 1,2
),

base_current AS (
SELECT 
        COALESCE(b.month,c.month,r.month) month,
        -- b.month,
        COALESCE(b.customer_id,c.customer_id,r.customer_id) customers,
        b.customer_id bcustomer_id,
        COALESCE(b.ltr_auto,'NEW') ltr_auto,
        COALESCE(b.recency_auto, 'NEW') recency_auto,
        c.customer_id acustomer_id,
        r.customer_id rcustomer_id,
        CASE 
        WHEN c.ao = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN c.ao = 0 AND c.fe > 0 THEN c.fe
        WHEN c.ao = 0 AND r.rr > 0 THEN r.rr
        WHEN c.ao IS NULL THEN 0
        ELSE c.ao END ao,
        
        CASE
        WHEN c.fe = 0 AND r.net_rides > 0 THEN r.net_rides 
        WHEN c.fe = 0 AND r.rr > 0 THEN r.rr
        WHEN c.fe IS NULL THEN 0
        ELSE c.fe END fe,
        
        CASE
        WHEN r.rr = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN r.rr IS NULL THEN 0
        ELSE r.rr END rr,
        
        COALESCE(r.net_rides, 0) net_rides,
        COALESCE(r.burn, 0) burn,
        COALESCE(r.subtotal, 0) subtotal

FROM base_month_segment AS b
FULL JOIN monthly_ao AS c ON b.customer_id = c.customer_id AND b.month = c.month 
FULL JOIN rr_net AS r ON b.customer_id = r.customer_id AND c.month = r.month
),

detailed_view AS 
(
SELECT 
        c.month,
        ltr_auto,
        case 
        when ltr_auto = 'LTR 1-4' then 1
        when ltr_auto = 'LTR 5-20' then 2
        when ltr_auto = 'LTR 21-50' then 3
        when ltr_auto = 'LTR 50+' then 4
        when ltr_auto = 'LTR >=5' then 5
        when ltr_auto = 'LTR =0' then 6
        when ltr_auto = 'NEW' then 7
        end row_order,
        recency_auto,
        COUNT(DISTINCT bcustomer_id) existing_custr_count,
        COUNT(DISTINCT CASE WHEN ao > 0 THEN acustomer_id END) ao_users,
        COUNT(DISTINCT CASE WHEN fe > 0 THEN acustomer_id END) fe_users,
        COUNT(DISTINCT CASE WHEN rr > 0 THEN rcustomer_id END) rr_users,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN rcustomer_id END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
        
FROM base_current c
GROUP BY 1,2,3,4
),

summary_view AS (
SELECT 
        c.month,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN c.customers END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
FROM base_current c
GROUP BY 1
),

a as 
(
SELECT  
        a.month,
        a.row_order,
        a.ltr_auto,
        a.recency_auto,
        a.existing_custr_count,
        COALESCE(try(a.existing_custr_count*100.00/b.existing_custr_total), 0) existing_custr_distr,
        a.ao_users,
        COALESCE(try(a.ao_users*100.00/a.existing_custr_count), 0) ao_conversion, 
        a.fe_users,
        COALESCE(try(a.fe_users*100.00/a.existing_custr_count), 0) fe_conversion, 
        a.rr_users,
        COALESCE(try(a.rr_users*100.00/a.existing_custr_count), 0) rr_conversion,
        COALESCE(try(a.rr_users*100.00/a.fe_users), 0) fe_rr,
        a.net_users,
        COALESCE(try(a.net_users*100.00/a.existing_custr_count), 0) net_conversion,
        COALESCE(try(a.net_users*100.00/a.rr_users), 0) rr_net,
        COALESCE(try(a.net_users*100.00/c.net_users), 0) net_users_distr,
        a.net_rides,
        COALESCE(try(a.net_rides*100.00/c.net_rides), 0) net_rides_distr,
        COALESCE(try(a.net_rides*1.00/a.net_users), 0) rpc,
        a.discount,
        a.subtotal
        
FROM detailed_view a
LEFT JOIN base_month_total b ON a.month = b.month
LEFT JOIN summary_view c ON a.month = c.month
ORDER BY 1,2,3
),

b as 
(
SELECT
        month,
        8 row_order,
        'TOTAL' ltr_auto,
        '' recency_auto,
        SUM(existing_custr_count) existing_custr_count,
        100.00 existing_custr_distr,
        SUM(ao_users) ao_users,
        COALESCE(try(SUM(ao_users)*100.00/SUM(existing_custr_count)), 0) ao_conversion,
        SUM(fe_users) fe_users,
        COALESCE(try(SUM(fe_users)*100.00/SUM(existing_custr_count)), 0) fe_conversion,
        SUM(rr_users) rr_users,
        COALESCE(try(SUM(rr_users)*100.00/SUM(existing_custr_count)), 0) rr_conversion,
        COALESCE(try(SUM(rr_users)*100.00/SUM(fe_users)), 0) fe_rr,
        SUM(net_users) net_users,
        COALESCE(try(SUM(net_users)*100.00/SUM(existing_custr_count)), 0) net_conversion,
        COALESCE(try(SUM(net_users)*100.00/SUM(rr_users)), 0) rr_net,
        100.00 net_users_distr,
        SUM(net_rides) net_rides,
        100.00 net_rides_distr,
        COALESCE(try(SUM(net_rides)*1.0/SUM(net_users)), 0) rpc,
        SUM(discount) discount,
        SUM(subtotal) subtotal
FROM a
GROUP BY 1
),

final as 
(
select * from a
union
select * from b
)
select 
        month,
        ltr_auto,recency_auto,
        existing_custr_count,existing_custr_distr,
        ao_users,ao_conversion,
        fe_users,fe_conversion,
        rr_users,rr_conversion,fe_rr "fe_rr(users)",
        net_users,net_conversion, rr_net "rr_net(users)", net_users_distr,
        net_rides,net_rides_distr,
        rpc,
        discount,
        subtotal,
        COALESCE(TRY(discount*100.0/subtotal),0) discount_perc,
        COALESCE(TRY(discount*1.0/net_rides),0) dpr
from final
order by 1,row_order,3

"""

In [ ]:
df_auto = pd.read_sql(auto, conn)

In [7]:
df_auto

,month,ltr_auto,recency_auto,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-10-01,LTR 1-4,1. 1_7_DAYS,441038,2.93,364237,82.59,343928,77.98,250319,...,41.25,72.69,6.10,422979,5.59,2.32,1010424.0,5.706852e+07,1.770545,2.388828
1,2022-10-01,LTR 1-4,2. 8_14_DAYS,331705,2.20,242153,73.00,224046,67.54,141241,...,26.53,62.30,2.95,175453,2.32,1.99,461595.0,2.387419e+07,1.933448,2.630876
2,2022-10-01,LTR 1-4,3. 15_21_DAYS,305717,2.03,208214,68.11,190980,62.47,114043,...,22.41,60.07,2.30,126389,1.67,1.84,361985.0,1.737730e+07,2.083091,2.864055
3,2022-10-01,LTR 1-4,4. 22_30_DAYS,365840,2.43,230926,63.12,210432,57.52,120901,...,19.13,57.87,2.34,123890,1.64,1.77,344982.0,1.712924e+07,2.013995,2.784583
4,2022-10-01,LTR 1-4,6. 31_45_DAYS,514111,3.41,297804,57.93,268697,52.26,146218,...,15.90,55.89,2.74,137246,1.81,1.68,411149.0,1.911654e+07,2.150750,2.995708
5,2022-10-01,LTR 1-4,7. 46_60_DAYS,540076,3.58,294431,54.52,262711,48.64,135176,...,13.61,54.39,2.46,118210,1.56,1.61,364348.0,1.668439e+07,2.183765,3.082210
6,2022-10-01,LTR 1-4,8. 61_90_DAYS,868365,5.76,425483,49.00,376402,43.35,175677,...,10.41,51.45,3.03,139985,1.85,1.55,419193.0,1.968408e+07,2.129604,2.994557
7,2022-10-01,LTR 1-4,9. 91_180_DAYS,1809109,12.00,695271,38.43,606504,33.53,252944,...,6.86,49.07,4.16,186379,2.46,1.50,559040.0,2.633804e+07,2.122557,2.999480
8,2022-10-01,LTR 1-4,99. 181_365_DAYS,1466169,9.73,397987,27.14,349503,23.84,120577,...,3.95,47.99,1.94,85203,1.13,1.47,282659.0,1.200501e+07,2.354509,3.317477
9,2022-10-01,LTR 1-4,999. 365_ABOVE,823296,5.46,171769,20.86,151121,18.36,43710,...,2.40,45.13,0.66,28851,0.38,1.46,115845.0,4.047786e+06,2.861935,4.015285


## LTR = 0

In [5]:
ltr_0 = f"""
WITH base_month_customer AS 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d') AS month,
    customer_id,
    --MAX(auto_lifetime_last_ride_date) 
    auto_lifetime_last_ride_date,
    auto_lifetime_first_ride_date,
    --MAX(auto_lifetime_rides) 
    auto_lifetime_rides

FROM datasets.iallocator_customer_segments
WHERE 
    run_date BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date)) , '%Y-%m-%d') 
    AND run_date = DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d')
    AND (auto_lifetime_rides > 0 or auto_lifetime_rides is not null or auto_lifetime_last_ride_date is not null)
--GROUP BY 1,2
),

ao_recent_ltr0 as
(
-- SELECT
--     customer_id,
--     max(day) ao_recency
-- FROM
    (
    SELECT 
        customerid customer_id,
        max(day) ao_recency
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)) - INTERVAL '365' DAY, '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        -- AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IN (SELECT DISTINCT customer_id FROM base_month_customer)-- where auto_lifetime_rides not between 1 and 10000)
        AND ao_sessions_unique_daily > 0
    GROUP BY 1
    )
-- GROUP BY 1
),

base_month_segment AS
(
SELECT * FROM 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    base_month_customer.customer_id,
    CASE
    WHEN auto_lifetime_rides IS NULL OR DATE(month) = DATE(auto_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN auto_lifetime_rides BETWEEN 1 AND 4 THEN '1-4'
    WHEN (auto_lifetime_rides BETWEEN 5 AND 20) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '5-20'
    WHEN (auto_lifetime_rides BETWEEN 21 AND 50) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '21-50'
    WHEN (auto_lifetime_rides > 50) AND (auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '50+'
    WHEN auto_lifetime_rides >= 5 THEN '>=5'
    ELSE 'LTR=0'
    END ltr_auto,
    
    CASE 
    WHEN auto_lifetime_last_ride_date IS NULL OR DATE(month) = DATE(auto_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN (auto_lifetime_rides BETWEEN 1 AND 4) AND auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '7' DAY) THEN '1. LAST_7_DAYS'
    WHEN (auto_lifetime_rides BETWEEN 1 AND 4) AND auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '14' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '8' DAY) THEN '2. 8_14_DAYS'
    WHEN (auto_lifetime_rides BETWEEN 1 AND 4) AND auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '21' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '15' DAY) THEN '3. 15_21_DAYS'
    WHEN (auto_lifetime_rides BETWEEN 1 AND 4) AND auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '22' DAY) THEN '4. 22_30_DAYS'
    WHEN (auto_lifetime_rides BETWEEN 1 AND 4) AND auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '45' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '5. 31_45_DAYS'
    WHEN (auto_lifetime_rides BETWEEN 1 AND 4) AND auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '60' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '46' DAY) THEN '6. 46_60_DAYS'
    WHEN (auto_lifetime_rides BETWEEN 1 AND 4) AND auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '61' DAY) THEN '7. 61_90_DAYS'    
    
    WHEN auto_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN '11. LAST_30_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '12. 31_90_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '8. 91_180_DAYS'
    WHEN auto_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '9. 181_365_DAYS'
    WHEN auto_lifetime_last_ride_date < (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) THEN '10. 365_ABOVE'
    ELSE 'LTR=0'
    END recency_auto,
    
    CASE
    WHEN DATE(ao_recency) >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '7' DAY) THEN '1. LAST_7_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '14' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '8' DAY) THEN '2. 8_14_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '15' DAY) THEN '3. 15_30_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '60' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '4. 31_60_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '61' DAY) THEN '5. 61_90_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '6. 91_180_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '7. 181_365_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '730' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '366' DAY) THEN '8.1_2_YEARS'
    ELSE 'NO_AO'
    END ao_recency
FROM base_month_customer 
LEFT JOIN ao_recent_ltr0 b on base_month_customer.customer_id = b.customer_id
-- where auto_lifetime_rides not between 1 and 10000
)
WHERE ltr_auto = 'LTR=0' AND recency_auto = 'LTR=0'
),

base_month_total AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    COUNT(DISTINCT customer_id) existing_custr_total 
FROM base_month_customer
GROUP BY 1
),

customer_rf_daily_kpi AS
(
SELECT 
    day,
    customerid,
    rr_sessions_unique_daily,
    net_rides_daily,
    ao_sessions_unique_daily,
    fe_sessions_unique_daily,
    discount_daily,
    subtotal_daily
        
FROM datasets.customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
    AND service_name = 'Auto'
    AND customerid IS NOT NULL
    AND customerid IN (SELECT DISTINCT customer_id FROM base_month_customer)-- where auto_lifetime_rides not between 1 and 10000)
),

rr_net AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customerid as customer_id,
    SUM(rr_sessions_unique_daily) rr,
    SUM(net_rides_daily) net_rides,
    SUM(discount_daily) burn,
    SUM(subtotal_daily) subtotal
FROM customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
GROUP BY 1,2
),

max_daily_ao AS
(
SELECT 
        day,
        customer_id,
        SUM(ao) AS ao,
        SUM(fe) AS fe
FROM
    (
    SELECT 
        customerid as customer_id,
        day,
        max(ao_sessions_unique_daily) AS ao,
        max(fe_sessions_unique_daily) AS fe
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IS NOT NULL
        AND customerid IN (SELECT DISTINCT customer_id FROM base_month_customer)-- where auto_lifetime_rides not between 1 and 10000)
    GROUP BY 1,2
    )
GROUP BY 1,2
),

monthly_ao AS
(
SELECT  
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customer_id,
    SUM(ao) AS ao,
    SUM(fe) AS fe
FROM max_daily_ao
GROUP BY 1,2
),

base_current AS (
SELECT 
        COALESCE(b.month,c.month,r.month) month,
        -- b.month,
        COALESCE(b.customer_id,c.customer_id,r.customer_id) customers,
        b.customer_id bcustomer_id,
        COALESCE(b.ltr_auto,'NEW') ltr_auto,
        COALESCE(b.recency_auto, 'NEW') recency_auto,
        ao_recency,
        c.customer_id acustomer_id,
        r.customer_id rcustomer_id,
        CASE 
        WHEN c.ao = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN c.ao = 0 AND c.fe > 0 THEN c.fe
        WHEN c.ao = 0 AND r.rr > 0 THEN r.rr
        WHEN c.ao IS NULL THEN 0
        ELSE c.ao END ao,
        
        CASE
        WHEN c.fe = 0 AND r.net_rides > 0 THEN r.net_rides 
        WHEN c.fe = 0 AND r.rr > 0 THEN r.rr
        WHEN c.fe IS NULL THEN 0
        ELSE c.fe END fe,
        
        CASE
        WHEN r.rr = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN r.rr IS NULL THEN 0
        ELSE r.rr END rr,
        
        COALESCE(r.net_rides, 0) net_rides,
        COALESCE(r.burn, 0) burn,
        COALESCE(r.subtotal, 0) subtotal

FROM base_month_segment AS b
FULL JOIN monthly_ao AS c ON b.customer_id = c.customer_id AND b.month = c.month 
FULL JOIN rr_net AS r ON b.customer_id = r.customer_id AND c.month = r.month
),

detailed_view AS 
(
SELECT 
        c.month,
        ltr_auto,
        case 
        when ltr_auto = '1-4' then 1
        when ltr_auto = '5-20' then 2
        when ltr_auto = '21-50' then 3
        when ltr_auto = '50+' then 4
        when ltr_auto = '>=5' then 5
        when ltr_auto = 'LTR=0' then 6
        when ltr_auto = 'NEW' then 7
        end row_order,
        recency_auto,
        ao_recency,
        COUNT(DISTINCT bcustomer_id) existing_custr_count,
        COUNT(DISTINCT CASE WHEN ao > 0 THEN acustomer_id END) ao_users,
        COUNT(DISTINCT CASE WHEN fe > 0 THEN acustomer_id END) fe_users,
        COUNT(DISTINCT CASE WHEN rr > 0 THEN rcustomer_id END) rr_users,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN rcustomer_id END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
        
FROM base_current c
GROUP BY 1,2,3,4,5
),

summary_view AS (
SELECT 
        c.month,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN c.customers END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
FROM base_current c
GROUP BY 1
),

a as 
(
SELECT  
        a.month,
        a.row_order,
        a.ltr_auto,
        a.recency_auto,
        ao_recency,
        a.existing_custr_count,
        COALESCE(try(a.existing_custr_count*100.00/b.existing_custr_total), 0) existing_custr_distr,
        a.ao_users,
        COALESCE(try(a.ao_users*100.00/a.existing_custr_count), 0) ao_conversion, 
        a.fe_users,
        COALESCE(try(a.fe_users*100.00/a.existing_custr_count), 0) fe_conversion, 
        a.rr_users,
        COALESCE(try(a.rr_users*100.00/a.existing_custr_count), 0) rr_conversion,
        COALESCE(try(a.rr_users*100.00/a.fe_users), 0) fe_rr,
        a.net_users,
        COALESCE(try(a.net_users*100.00/a.existing_custr_count), 0) net_conversion,
        COALESCE(try(a.net_users*100.00/a.rr_users), 0) rr_net,
        COALESCE(try(a.net_users*100.00/c.net_users), 0) net_users_distr,
        a.net_rides,
        COALESCE(try(a.net_rides*100.00/c.net_rides), 0) net_rides_distr,
        COALESCE(try(a.net_rides*1.00/a.net_users), 0) rpc,
        a.discount,
        a.subtotal
        
FROM detailed_view a
LEFT JOIN base_month_total b ON a.month = b.month
LEFT JOIN summary_view c ON a.month = c.month
ORDER BY 1,2,3
),

final as 
(
select * from a
)
select 
        month,
        ltr_auto,recency_auto,ao_recency,
        existing_custr_count,existing_custr_distr,
        ao_users,ao_conversion,
        fe_users,fe_conversion,
        rr_users,rr_conversion,fe_rr "fe_rr(users)",
        net_users,net_conversion, rr_net "rr_net(users)", net_users_distr,
        net_rides,net_rides_distr,
        rpc,
        discount,
        subtotal,
        COALESCE(TRY(discount*100.0/subtotal),0) discount_perc,
        COALESCE(TRY(discount*1.0/net_rides),0) dpr
from final
order by 1,row_order,4

"""

In [ ]:
df_ltr_0 = pd.read_sql(ltr_0, conn)

In [8]:
df_ltr_0

,month,ltr_auto,recency_auto,ao_recency,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-10-01,LTR=0,LTR=0,1. LAST_7_DAYS,712445,4.73,567104,79.60,512222,71.90,...,16.53,48.88,4.92,206688,3.15,1.76,1482988.0,28415567.0,5.218928,7.175008
1,2022-10-01,LTR=0,LTR=0,2. 8_14_DAYS,305637,2.03,175827,57.53,148297,48.52,...,8.06,44.36,1.03,37604,0.57,1.53,331704.0,5287253.0,6.273655,8.820976
2,2022-10-01,LTR=0,LTR=0,3. 15_30_DAYS,485813,3.22,217449,44.76,178420,36.73,...,6.14,44.72,1.25,45091,0.69,1.51,401788.0,6498648.0,6.182640,8.910603
3,2022-10-01,LTR=0,LTR=0,4. 31_60_DAYS,596948,3.96,173676,29.09,140725,23.57,...,3.78,44.13,0.94,34568,0.53,1.53,293783.0,4983409.0,5.895222,8.498698
4,2022-10-01,LTR=0,LTR=0,5. 61_90_DAYS,383765,2.55,70544,18.38,57680,15.03,...,2.37,45.28,0.38,14024,0.21,1.54,121147.0,2020703.0,5.995290,8.638548
5,2022-10-01,LTR=0,LTR=0,6. 91_180_DAYS,725216,4.81,79391,10.95,68803,9.49,...,1.55,45.48,0.47,17794,0.27,1.58,150243.0,2551308.0,5.888862,8.443464
6,2022-10-01,LTR=0,LTR=0,7. 181_365_DAYS,621846,4.13,37227,5.99,33646,5.41,...,0.92,48.26,0.24,9140,0.14,1.60,77513.0,1298302.0,5.970337,8.480635
7,2022-10-01,LTR=0,LTR=0,NO_AO,444842,2.95,19750,4.44,17809,4.00,...,1.03,59.61,0.19,7838,0.12,1.72,65104.0,1074115.0,6.061176,8.306201
8,2022-10-01,NEW,NEW,None,0,0.00,5717537,0.00,5265850,0.00,...,0.00,66.63,90.58,6198896,94.33,2.86,13457292.0,812110492.0,1.657076,2.170918
